<a href="https://colab.research.google.com/github/SeptiLutfiana/MESIN-LEARNING/blob/main/Pertemuan10_Praktikum2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generator Teks dengan RNN

Import TensorFlow

In [7]:
import tensorflow as tf
import numpy as np
import os
import time

In [8]:
# download data set
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [9]:
# load data
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [10]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [11]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


Olah Teks

In [12]:
example_texts=['abcdefg','xyz']
chars=tf.strings.unicode_split(example_texts,input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

In [13]:
ids_from_chars=tf.keras.layers.StringLookup(
vocabulary=list(vocab),mask_token=None)

In [14]:
ids=ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [15]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [16]:
chars=chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [17]:
tf.strings.reduce_join(chars,axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [18]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids),axis=-1)

Prediksi

In [19]:
all_ids=ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [20]:
ids_dataset=tf.data.Dataset.from_tensor_slices(all_ids)

In [21]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [22]:
seq_length=100

In [23]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [24]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [25]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text,target_text

In [26]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [27]:
dataset=sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:",text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

In [29]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Buat Model

In [30]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [31]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [32]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Uji Model

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [34]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
sampled_indices=tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices=tf.squeeze(sampled_indices,axis=-1).numpy()

In [36]:
sampled_indices

array([37,  1, 34, 29, 56, 13, 27, 24,  7, 21, 14, 26, 28, 42, 44, 17, 62,
       10, 49, 35, 60, 42, 10,  2, 64, 58, 19, 11, 57, 41, 29, 52, 25, 14,
       39, 14, 12,  1, 33, 52,  2, 30, 22, 47, 16, 18, 33, 31, 27, 52,  4,
       21, 15, 17, 11, 33, 38,  7,  8,  8, 25, 61, 21, 55, 51, 13, 39, 32,
       41, 62, 25, 11, 26,  6, 56, 62, 36, 59,  6,  7, 37,  4, 29, 47, 23,
       31, 26, 17, 12, 28, 28, 39,  1,  7, 42, 50, 56, 39, 45, 42])

In [37]:
print("Input:\n",text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n",text_from_ids(sampled_indices).numpy())

Input:
 b"s to do here, Thomas tapster? let's withdraw.\n\nPOMPEY:\nHere comes Signior Claudio, led by the provos"

Next Char Predictions:
 b"X\nUPq?NK,HAMOceDw3jVuc3 ysF:rbPmLAZA;\nTm QIhCETRNm$HBD:TY,--LvHpl?ZSbwL:M'qwWt',X$PhJRMD;OOZ\n,ckqZfc"


Train Model

In [38]:
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.191628, shape=(), dtype=float32)


In [40]:
tf.exp(example_batch_mean_loss).numpy()

66.13036

In [41]:
model.compile(optimizer='adam',loss=loss)

In [42]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [43]:
EPOCHS=20

In [44]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=
 [checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 51ms/step - loss: 2.7395
Epoch 2/20
172/172 [==============================] - 11s 50ms/step - loss: 2.0043
Epoch 3/20
172/172 [==============================] - 11s 50ms/step - loss: 1.7291
Epoch 4/20
172/172 [==============================] - 11s 51ms/step - loss: 1.5650
Epoch 5/20
172/172 [==============================] - 10s 51ms/step - loss: 1.4645
Epoch 6/20
172/172 [==============================] - 11s 52ms/step - loss: 1.3940
Epoch 7/20
172/172 [==============================] - 12s 52ms/step - loss: 1.3418
Epoch 8/20
172/172 [==============================] - 11s 53ms/step - loss: 1.2967
Epoch 9/20
172/172 [==============================] - 11s 54ms/step - loss: 1.2563
Epoch 10/20
172/172 [==============================] - 11s 54ms/step - loss: 1.2176
Epoch 11/20
172/172 [==============================] - 11s 54ms/step - loss: 1.1791
Epoch 12/20
172/172 [==============================] - 11s 55ms/step - loss: 1.1391
E

Generate Teks

In [45]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [46]:
one_step_model=OneStep(model,chars_from_ids,ids_from_chars)

In [47]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Tybalt, believe this, that my foul way to be one.

POMPEY:
Pray, sir, what hath bestock me? I speak too Kate:
I have some weapons that a careful store
Of that, and matter well owes;
Which, being that, by great ones all itself.
Ah, traitors farewell and heart, thy sorrow,
That you be accused and too much
wrinking paint.
Come, come, dame, peace: thou art too furlones,
Nor with a birth and notice of thine;
For in this bost from me; thou art no brother's use:
I'll have this, vowsor to what ends at hand:
And all to him in heaven be out, and hearers as
she is my son, and my process shore, bearing
A place of water in the law doth learn,
And let my foot can sink them o'.
I thought, as being assurance sped time
With thine own toward till I have: I have
here her, sir, that she be both of twelve,
I do these worth of all his deeds,
Not by my spider, that have I have the daith
hereries are obsed and fair and inquired for some it:
Bring me well, I had too long have:
My trestle-moning. I cunni

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nWhy, then he hath done many to be much upon my house,\nYou have neither distribute my heart!\nA countenance of those grounds to't aid them.\nWhy, how now, known love to the nouth!\nSo whom you seek together, dancing,\nspeaking, no greenion banished.\nAre madness, my looring? is he master\nThan Bolingbroke take up more strong thunder?\nMust yet thou lovest not in the sun\nAnd thou dost give thee so, my little bid\nHave I did soil the choler in 'em; and, Master\nEarth his good will to do some mine art thou,\nLady for beasts, devover back and stood,\nScarcist well becomes the heavy help,\nAnd he hath sorrow then both publicle,\nThou rough Bohemio in beauty's mine,\nSure thou the vengeance plucks it bitterly;\nOr, if thou becomes strong Angelo.\n\nDUKE OF YORK:\nHo! who is not in these lamentation;\nIf you'll tell the people who comes wear it.\n\nGRUMIO:\nSir, your ancury word of these,\nThat brought it after their demands,\nRetirest that they there should be contente

Ekspor Model Generator

In [49]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [50]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
I'll tell thee as Polixenes;
With witust commanding thus.

HERMIONE:
STir!

GRUMIO:
I am any toible
